# Bir CSV dosyasından içerisinden İptal Edilen Toplam Satış Miktarını Bulma

In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
import os

In [2]:
pyspark = SparkSession.builder \
.master("local[4]") \
.appName("Create a RDD") \
.config("spark.executor.memory", "4g") \
.config("spark.driver.memory", "2g") \
.getOrCreate()

23/04/14 20:04:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
sc = pyspark.sparkContext
sc.setLogLevel("error")

In [9]:
file = f"{os.getcwd()}/OnlineRetail.csv"
retailRDD = sc.textFile(file)
retailRDD.take(5)

['InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country',
 '536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom']

In [27]:
header = retailRDD.first()
headerRDD = sc.parallelize(header)
retailRDD_without_header = retailRDD.filter(lambda x: x != header)
retailRDD_without_header.take(5)

['536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84029E;RED WOOLLY HOTTIE WHITE HEART.;6;1.12.2010 08:26;3,39;17850;United Kingdom']

In [28]:
def cancelled_price(line: str):
    is_cancelled = True if (line.split(";")[0].startswith("C")) else False
    quantity = float(line.split(";")[3])
    price = float(line.split(";")[5].replace(",", "."))
    total = quantity * price
    
    return  (is_cancelled, total)

In [29]:
cancelled_total = retailRDD_without_header.map(cancelled_price)

In [30]:
cancelled_total.take(5)

[(False, 15.299999999999999),
 (False, 20.34),
 (False, 22.0),
 (False, 20.34),
 (False, 20.34)]

In [31]:
cancelled_total_reduced = cancelled_total.reduceByKey(lambda x,y: x+y)
cancelled_total_reduced.take(5)

[(False, 10644560.424000263), (True, -896812.4900000116)]

In [32]:
cancelled_total_reduced_only_true = cancelled_total_reduced.filter(lambda x: x[0] == True)
cancelled_total_reduced_only_true.take(5)

[(True, -896812.4900000116)]

In [33]:
cancelled_total_reduced.filter(lambda x: x[0] == True).map(lambda x: x[1]).take(1)

[-896812.4900000116]

In [36]:
cancelled_total_reduced.filter(lambda x: x[0] == True).values().take(1)

[-896812.4900000116]